In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

In [2]:
# Khởi tạo trình duyệt Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized") 
options.add_argument("--disable-blink-features=AutomationControlled")  
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [4]:
driver.get("https://www.livecareer.com/cv/examples")
time.sleep(5)  # Wait for initial page load
print(driver.page_source)

<html lang="en-US"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1"><meta http-equiv="X-UA-Compatible" content="ie=edge"> <script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-971063891&amp;l=dataLayer&amp;cx=c&amp;gtm=45je53j1v871411692za200zb77769872&amp;tag_exp=102482433~102788824~102803279~102813109~102926061"></script><script src="//bat.bing.com/bat.js" async=""></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-TEPPBW8QXP&amp;l=dataLayer&amp;cx=c&amp;gtm=45He53j1v77769872za200&amp;tag_exp=102308675~102482433~102788824~102803279~102813109"></script><script type="text/javascript" defer="" src="https://cdn.segment.com/analytics.js/v1/B7rLk975WsvRuE8E91XEVqzL3jn8HddD/analytics.min.js"></script><script type="text/javascript" src="https://cse.expertrec.com/js/ci_customSearch_separate.1742557672350.min.js"></script><script type="text/javascript" 

In [6]:
# Cuộn xuống để tải toàn bộ nội dung (Lazy Load)
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [11]:
category_elements = driver.find_elements(By.CSS_SELECTOR, "h3.ind-heading")
data = []

for category_element in category_elements:
    try:
        category = category_element.text.strip()
        print(f"📌 Processing category: {category}")

        # Tìm section cha gần nhất chứa category và các ảnh CV
        parent_section = category_element.find_element(By.XPATH, "./ancestor::section")
        cv_elements = parent_section.find_elements(By.CLASS_NAME, "mini-resume.add-img")

        for cv in cv_elements:
            try:
                # Tìm tất cả các ảnh trong mini-resume (page-1, page-2, v.v.)
                img_elements = cv.find_elements(By.CSS_SELECTOR, "img.lazy.lazy-hidden")

                # Tạo dictionary để lưu các link theo trang
                pages = {}
                for idx, img_element in enumerate(img_elements, start=1):
                    # Lấy link ảnh từ data-lazy-src
                    img_link = img_element.get_attribute("data-lazy-src")

                    # Nếu không có data-lazy-src, đợi src tải
                    if not img_link or "data:image" in img_link:
                        WebDriverWait(driver, 10).until(
                            lambda d: "data:image" not in img_element.get_attribute("src")
                        )
                        img_link = img_element.get_attribute("src")

                    # Kiểm tra link ảnh hợp lệ
                    if not img_link or "data:image" in img_link:
                        print(f"⚠ {category} - Invalid image link for page {idx}: {img_link}")
                        continue

                    # Lưu link vào dictionary với key là số trang
                    pages[f"page_{idx}"] = img_link
                    print(f"✔ {category} - Page {idx}: {img_link}")

                # Thêm cặp link vào danh sách dữ liệu
                # Nếu không có page-2, để trống cột Page_2_Link
                page_1_link = pages.get("page_1", "")
                page_2_link = pages.get("page_2", "")
                data.append([category, page_1_link, page_2_link])

            except Exception as e:
                print(f"❌ Error processing CV in category {category}: {e}")

    except Exception as e:
        print(f"❌ Error processing category section: {e}")

📌 Processing category: Accounting CV Examples
✔ Accounting CV Examples - Page 1: https://www.livecareer.com/lcapp/uploads/2023/02/accountant-example-CV-page-1.svg
✔ Accounting CV Examples - Page 2: https://www.livecareer.com/lcapp/uploads/2023/02/accountant-example-CV-page-2.svg
✔ Accounting CV Examples - Page 1: https://www.livecareer.com/lcapp/uploads/2023/02/executive-assistant-example-CV-page-1.svg
✔ Accounting CV Examples - Page 2: https://www.livecareer.com/lcapp/uploads/2023/02/executive-assistant-example-CV-page-2.svg
✔ Accounting CV Examples - Page 1: https://www.livecareer.com/lcapp/uploads/2023/02/payroll-specialist-example-CV-page-1.svg
✔ Accounting CV Examples - Page 2: https://www.livecareer.com/lcapp/uploads/2023/02/payroll-specialist-example-CV-page-2.svg
✔ Accounting CV Examples - Page 1: https://www.livecareer.com/lcapp/uploads/2023/02/operations-manager-example-CV-page-1.svg
✔ Accounting CV Examples - Page 2: https://www.livecareer.com/lcapp/uploads/2023/02/operation

In [12]:
# Đóng trình duyệt
driver.quit()

In [17]:
import csv
csv_file = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/cv1_links.csv"
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Category", "Page_1_Link", "Page_2_Link"])  # Tiêu đề cột mới
    writer.writerows(data)

print(f"✅ Đã lưu dữ liệu vào {csv_file}")

✅ Đã lưu dữ liệu vào D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/cv1_links.csv


In [18]:
import pandas as pd

# Đọc file CSV
csv_file = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/cv1_links.csv"
df = pd.read_csv(csv_file)

# Xử lý cột Category: Xóa "CV Examples" khỏi giá trị
df['Category'] = df['Category'].str.replace(' CV Examples', '', regex=False)

# Lưu lại vào file CSV mới
output_file = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/cv1_links_cleaned.csv"
df.to_csv(output_file, index=False)

print(f"✅ Đã xử lý và lưu dữ liệu vào {output_file}")

✅ Đã xử lý và lưu dữ liệu vào D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/cv1_links_cleaned.csv
